In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

In [2]:
diamonds = pd.read_excel("/SBA/diamonds.xlsx",
                        sheet_name = 0,
                        header     = 0)

- - -

### 정규성 검정(Normality Test)

In [3]:
# 귀무가설 : 모집단의 양적 자료는 정규분포를 따른다.
# 대립가설 : 모집단의 양적 자료는 정규분포를 따르지 않는다.

In [4]:
# 1. Shapiro-Wilk Normalitytest
# 데이터의 개수가 5000개 미만일 때 가장 유명함
# scipy.stats -> stats
# stats.shapiro(data.variable)
stats.shapiro(diamonds.price)

C:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:1681: UserWarning: p-value may not be accurate for N > 5000.
  warnings.warn("p-value may not be accurate for N > 5000.")


ShapiroResult(statistic=0.7981084585189819, pvalue=0.0)

In [5]:
# 2. Anderson-Darling Normality test
# 데이터의 개수가 5000개 이상일 때 가장 유명함
# stats.anderson(data.variable, dist = "norm")

# 참고
# dist : distribution        = 분포
# norm : normal distribution = 정규 분포

# 표본에 있는 데이터의 개수(표본 크기) = sample size
stats.anderson(diamonds.price, dist = "norm")

AndersonResult(statistic=3474.0163510249695, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

결론  
유의수준 0.05에서의 임계값이 0.787이고  
표본에서 검정통계량이 3474.016 으로 임계값 0.787 보다 크므로  
price는 정규분포를 따르지 않는다. 즉, 정규성 가정이 깨짐

- - -

#### 일표본 검정의 단계

Step 1. 정규성 검정(Normality Test)
- stats.shapiro or stats.anderson 사용
- 귀무가설 : 정규분포를 따른다.
- 대립가설 : 정규분포를 따르지 않는다

Step 2-1. 일표본 t검정(One sample t-test)  
**1단계의 결론이 귀무가설(정규성 만족) 일 때**  
stats.ttest_1samp(data.variable, popmean = )

Step 2-2. 윌콕슨의 부호 순위 검정(Wilcox's signed rank test)  
**조건 : 1단계의 결론이 대립가설(정규성 불만족) 일 때**
stats.wilcoxon(data.variable - myu, alternative = )

In [6]:
# 연습 : diamonds의 price
# 귀무가설 : 다이아몬드 가격의 평균은 4000달러이다.
# 대립가설 : 다이아몬드 가격의 평균은 4000달러보다 많다.
diamonds.price.count()

53940

In [7]:
# 데이터 개수가 5000개 이상 - 정규성 불만족
stats.anderson(diamonds.price, dist = "norm")

AndersonResult(statistic=3474.0163510249695, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

In [8]:
stats.wilcoxon(diamonds.price - diamonds.price.mean(), alternative = "greater")

WilcoxonResult(statistic=576399622.0, pvalue=1.0)

In [9]:
# 결론
# 유의확률이 1.000이므로 유의수준 0.05에서
# 다이아몬드 가격은 통계적으로 유의하게 변하지 않았다.

- - -

## 독립 2표본 검정(Two sample test)
- 두 개의 독립적인 모집단의 양적 자료가 차이가 있는지를 분석하는 방법
- 독립 : A 집단의 양적 자료가 B 집단의 양적 자료에게 영향을 주지 않는 상태

### 1. 독립 2표본 t검정(Two sample t-test)

* 1.1 등분산이 가정된 독립 2표본 t검정
* 1.2 이분산이 가정된 독립 2표본 t검정

In [18]:
# 귀무가설 : Ideal 가격의 평균과 NonIdeal 가격의 평균은 같다.
# 대립가설 : Ideal 가격의 평균이 NonIdeal 가격의 평균보다 비싸다.

ideal    = diamonds.loc[diamonds.cut == "Ideal", "price"]
nonideal = diamonds.loc[diamonds.cut != "Ideal", "price"]

In [19]:
# 등분산 검정(Equality of Variance Test)
# 귀무가설 : 등분산이다.
# 대립가설 : 이분산이다.

# (1) Levene  Test : stats.levene(data1, data2)
# (2) Bartlett Test : stats.bartlett(data1, data2)

In [20]:
# levene의 등분산 검정
stats.levene(ideal, nonideal)

LeveneResult(statistic=244.8982065343527, pvalue=4.446981310047094e-55)

In [21]:
# 결론
# 244.898 = 검정통계량 = F
# F ~ F 분포
# p-value : 0.000

# 유의확률이 0.000 이므로 유의수준 0.05 에서 이분산이다.

In [24]:
# Bartlett의 등분산 검정
stats.bartlett(ideal, nonideal)

BartlettResult(statistic=117.27579363247222, pvalue=2.4980059769203218e-27)

- - -

In [22]:
# 이분산이 가정된 독립 2표본 검정
# stats.ttest_ind(data1, data2, equal_var = False)

# 참고
# ind        : independent
# equal_var  : equality of variance

stats.ttest_ind(ideal, nonideal, equal_var = False)

Ttest_indResult(statistic=-22.985734120999393, pvalue=2.7359864985506295e-116)

In [25]:
# statistic = -22.9857... = 이분산 일 때의 t 값

# 유의확률이 1.000이므로 유의수준 0.05에서
# 품질이 Ideal인 다이아몬드와 품질이 NonIdeal의 가격에
# 통계적으로 유의한 차이는 없는 것으로 나타났다.

 - - -

## 2. 윌콕슨의 순위합 검정(Wilcoxon's rank sum test)

In [26]:
# stats.ranksums(data1, data2)
stats.ranksums(ideal, nonideal)

RanksumsResult(statistic=-27.277266218787283, pvalue=7.894328467663634e-164)

**결론**

-27.277 = Z : 표준 정규분포  
p-value = 1 - (0.000) / 2 = 1.000  
  
유의확률이 1.000 이므로 유의수준 0.05에서  
Ideal과 NonIdeal의 가격에 통계적으로 유의한 차이는 없는 것으로 나타났다.

### 단계
* 1단계 : 정규성 검정 : 2번 실시
 * 귀무가설 : 정규분포를 따른다.
 * 대립가설 : 정규분표를 따르지 않는다.

In [27]:
# ideal
stats.anderson(ideal, dist = "norm")

AndersonResult(statistic=1865.0905557672413, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

**결론**  
유의수준        : 0.05  
임계값 = 기각역 : 0.787  
검정통계량      : 1865.091  
  
검정통계량(1865.091)이 기각역(0.787)보다 크므로 대립가설 : 정규성 가정이 깨짐

In [28]:
# (2) NonIdeal
stats.anderson(nonideal, dist = "norm")

AndersonResult(statistic=1725.841416197174, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))

**결론**  
유의수준        : 0.05  
임계값 = 기각역 : 0.787  
검정통계량      : 1725.841 
  
검정통계량(1725.841)이 기각역(0.787)보다 크므로 대립가설 : 정규성 가정이 깨짐

In [29]:
# 2단계 : Wilcoxon's rank sum test
stats.ranksums(ideal, nonideal)

RanksumsResult(statistic=-27.277266218787283, pvalue=7.894328467663634e-164)

- - -

In [30]:
# 문제1
# cut : "Fair", "Good", "Very good", "Premium", "Ideal"

# "good" data, "very_good" data 선언
# 귀무가설 : "Good" 가격의 평균과 "Very Good" 가격의 평균은 같다.
# 대립가설 : "Good" 가격의 평균과 "Very Good" 가격의 평균은 같지 않다.

In [37]:
good      = diamonds.loc[diamonds.cut == "Good", "price"]
very_good = diamonds.loc[diamonds.cut == "Very Good", "price"]

In [44]:
# 1단계 : 정규성 검정
# 귀무가설 : 정규분포를 따른다.
# 대립가설 : 정규분포를 따르지 않는다.

good_normality      = stats.shapiro(good)
very_good_normality = stats.anderson(very_good, dist = "norm")

print("Good : ", good_normality)
print("Very good : ", very_good_normality)

Good :  ShapiroResult(statistic=0.8170838952064514, pvalue=0.0)
Very good :  AndersonResult(statistic=673.1474634497645, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))


In [ ]:
# 결론
# Good      : 검정통계량 = 0.817   / p-value = 0.000 --> 정규성 가정이 깨짐  
# Very Good : 검정통계량 : 673.147 / 기각역(0.787)   --> 정규성 가정이 깨짐

In [42]:
# 2단계 : Wilcoxon's rank sum test
stats.ranksums(good, very_good)

RanksumsResult(statistic=2.2734929145300424, pvalue=0.022996492627487226)

In [45]:
# 결론
# 검정통계량 : 2.273
# p-value    : 0.023

# 유의확률(p-value)이 0.023 이므로 유의수준 0.05 에서 
# "Good" 과 "Very Good" 의 가격에는 통계적으로 유의한 차이가 있는 것으로 나타났다.

- - -

In [46]:
# 문제 2
# carat_07_less    : carat이 0.7 미만인 데이터의 price
# carat_07_greater : carat이 0.7 이상인 데이터의 price

# 기무가설 : carat_07_less 가격의 평균과 carat_07_greater 가격의 평균은 같다.
# 대립가설 : carat_07_less 가격의 평균이 carat_07_greater 가격의 평균보다 작다.

In [48]:
carat_07_less    = diamonds.loc[diamonds.carat <  0.7, "price"]
carat_07_greater = diamonds.loc[diamonds.carat >= 0.7, "price"]

In [52]:
# 1단계 : 정규성 검정
# 귀무가설 : 정규분포를 따른다.
# 대립가설 : 정규분포를 따르지 않는다.

less_normality  = stats.anderson(carat_07_less,    dist = "norm")
great_normality = stats.anderson(carat_07_greater, dist = "norm")

print("Less   : ", less_normality)
print("Greater: ", great_normality)

Less   :  AndersonResult(statistic=829.0496819120162, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))
Greater:  AndersonResult(statistic=1231.4303477231406, critical_values=array([0.576, 0.656, 0.787, 0.918, 1.092]), significance_level=array([15. , 10. ,  5. ,  2.5,  1. ]))


In [53]:
# 결론
# less    : 검정통계량 = 829.050  / 기각역(0.787)  --> 정규성 가정이 깨짐  
# greater : 검정통계량 : 1231.43  / 기각역(0.787)  --> 정규성 가정이 깨짐

In [54]:
# 2단계 : Wilcoxon's rank sum test
stats.ranksums(carat_07_less, carat_07_greater)

RanksumsResult(statistic=-198.14102870691278, pvalue=0.0)

In [ ]:
# 결론
# 검정통계량 : Z = -198.141
# p-value    : 0.000(python의 p-value) / 2 = 0.000

# 유의확률(p-value)이 0.000 이므로 유의수준 0.05 에서 
# "Less" 과 "Greater" 의 가격에는 통계적으로 유의하게 작은 것으로 나타났다.